In [1]:
import numpy as np

In [2]:
input = """\
###..#.##.####.##..###.#.#..
#..#..###..#.......####.....
#.###.#.##..###.##..#.###.#.
..#.##..##...#.#.###.##.####
.#.##..####...####.###.##...
##...###.#.##.##..###..#..#.
.##..###...#....###.....##.#
#..##...#..#.##..####.....#.
.#..#.######.#..#..####....#
#.##.##......#..#..####.##..
##...#....#.#.##.#..#...##.#
##.####.###...#.##........##
......##.....#.###.##.#.#..#
.###..#####.#..#...#...#.###
..##.###..##.#.##.#.##......
......##.#.#....#..##.#.####
...##..#.#.#.....##.###...##
.#.#..#.#....##..##.#..#.#..
...#..###..##.####.#...#..##
#.#......#.#..##..#...#.#..#
..#.##.#......#.##...#..#.##
#.##..#....#...#.##..#..#..#
#..#.#.#.##..#..#.#.#...##..
.#...#.........#..#....#.#.#
..####.#..#..##.####.#.##.##
.#.######......##..#.#.##.#.
.#....####....###.#.#.#.####
....####...##.#.#...#..#.##."""

asteroids = np.array([[c == '#' for c in line] for line in input.splitlines()])

In [3]:
X, Y = np.indices(asteroids.shape)
visible = np.zeros(asteroids.shape, dtype=int)

for (x, y), ast in np.ndenumerate(asteroids):
    if ast:
        mask = asteroids.copy()
        mask[x, y] = False

        distX = -X + x
        distY = Y - y

        ang = np.around(np.angle(distX+distY*1j, deg=True), 2)
        ang[ang < 0] += 360
        ang[~mask] = np.nan

        visible[x, y] = len(np.unique(ang[mask]))

max_visible = np.max(visible)
best_location = np.argwhere(visible == max_visible).flatten()
print(max_visible, best_location)

282 [19 22]


In [4]:
x, y = tuple(best_location)

def new_map(asteroids, x, y):
    mask = asteroids.copy()
    mask[x, y] = False

    X, Y = np.indices(asteroids.shape)
    distX = -X + x
    distY = Y - y

    ang = np.around(np.angle(distX+distY*1j, deg=True), 2)
    ang[ang < 0] += 360
    ang[~mask] = np.nan

    abs = np.around(np.abs(distX+distY*1j), 2)
    abs[~mask] = np.nan

    return ang, abs

it = 0
lasered = 0
found = False

while not found:
    ang, abs = new_map(asteroids, x, y)

    smallest_angle = -1
    while not found:
        smallest_angle   = np.nanmin(ang[ang > smallest_angle])
        astroids_on_line = np.where(ang == smallest_angle)
        nearest_on_line  = np.argmin(abs[astroids_on_line])
        nearest_x        = astroids_on_line[0][nearest_on_line]
        nearest_y        = astroids_on_line[1][nearest_on_line]

        asteroids[nearest_x, nearest_y] = False
        lasered += 1

        if lasered == 200:
            print(nearest_x + 100 * nearest_y)
            found = True
            break

        if np.all(ang[asteroids] <= smallest_angle):
            break
    if np.count_nonzero(asteroids) <= 1:
        break

1008
